In [25]:
# Imports
import matplotlib.pyplot as plt 
import matplotlib.image as mtimage 
import numpy as np 
import cv2
from collections import deque 


%matplotlib inline 
print(np.__version__)

1.22.4


In [63]:
class Line:
    def __init__(self,x1,y1,x2,y2):
        self.x1 = np.float32(x1)
        self.x2 = np.float32(x2) 
        self.y1 = np.float32(y1) 
        self.y2 = np.float32(y2)
        self.slope = self.compute_slope()
        self.intercept = self.compute_intercept()
        
    def get_coords(self):
        return [self.x1,self.y1,self.x2,self.y2]
    
    def compute_slope(self):
        return (self.y2-self.y1)/(self.x2-self.x1 + np.finfo(float).eps)
    
    def compute_intercept(self):
        return self.y1 - self.slope * self.x1
    
    def draw(self,image):
        color = (0,0,255)
        thickness = 10 
        mod_value = 10**5
#         print(self.x1%mod_value, self.y1%mod_value,self.x2%mod_value, self.y2%mod_value)
        cv2.line(image,(int(self.x1%mod_value), int(self.y1%mod_value)), (int(self.x2%mod_value), int(self.y2%mod_value)), color, thickness)
        




In [67]:


def hough_transformation(image):
    # Define Parameter 
    rho = 2 
    theta = np.pi / 180
    threshold = 1
    min_line_len = 15
    max_line_gap = 5
    # Return list of lines detected 
    return cv2.HoughLinesP(image,rho,theta,threshold,np.array([]),minLineLength=min_line_len,maxLineGap=max_line_gap)

def compute_lanes_from_candidates(candidates,shape):
    left_fit = [] 
    right_fit = [] 
    for line in candidates:
        if line.slope < 0 :
            left_fit.append(line)
        elif line.slope > 0 :
            right_fit.append(line)
            
    left_intercept = np.median([line.intercept for line in left_fit]).astype(int)
    left_slope = np.median([line.slope for line in left_fit ])
    # Left Coordinates 
    x1,y1 = 0, left_intercept
    x2,y2 = -np.int32(np.round(left_intercept/left_slope)),0
    left_lane = Line(x1,y1,x2,y2)
    
    right_intercept = np.median([line.intercept for line in right_fit]).astype(int)
    right_slope = np.median([line.slope for line in right_fit ])
    # Right Coordinates 
    x1,y1 = 0, right_intercept
    x2,y2 = np.int32(np.round((shape[0]-right_intercept)/right_slope)),shape[0]
    right_lane = Line(x1,y1,x2,y2)
    return left_lane,right_lane
    
def smoothen_the_lines(lane_lines):
#     print(lane_lines)
    size = len(lane_lines)
    avg_left = np.zeros((size,4))
    avg_right = np.zeros((size,4))
    
    for index in range(size):
        avg_left[index] += lane_lines[index][0].get_coords()
        avg_right[index] += lane_lines[index][1].get_coords()
        
    avg_left_lane = Line(*np.mean(avg_left, axis=0))
    avg_right_lane = Line(*np.mean(avg_right, axis=0))
    
    return avg_left_lane, avg_right_lane
    
def get_lanelines(frame,shape):
    
    frame = cv2.resize(frame,(shape[1],shape[0]))
    gray_image = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    
    kernel_size = 17
    blue_image = cv2.GaussianBlur(gray_image,(kernel_size,kernel_size),0)
    
    low_threshold = 50
    high_threshold = 80
    canny_image = cv2.Canny(blue_image,low_threshold,high_threshold)
    
    detected_lines = hough_transformation(canny_image)
    detected_lines = [Line(line[0][0],line[0][1],line[0][2],line[0][3]) for line in detected_lines]
    
    candidate_lines = [] 
    for line in detected_lines:
        if 0.5 <= np.abs(line.slope) <= 2.0:
            candidate_lines.append(line)
    lane_lines = compute_lanes_from_candidates(candidate_lines,shape)
    return lane_lines
    
def region_of_interest(image):
    mask = np.zeros_like(image)
    mask_color = (0,0,255)
    shape = image.shape
    vertices = np.array([[(100,shape[0]),(460,320),(540,320),(shape[1]-50,shape[0])]])
    cv2.fillPoly(mask,vertices,mask_color)
    masked_image = cv2.bitwise_and(image,mask)
    
    return masked_image 
    
    

# Algorithm for Lane Detection in an image 
def process_image(buffer,shape):
    
    # Get the Lane lines for frame buffer or Get the lane lines for current image and last nine images 
    # which can then be used to find the average coordinates for drawing lane lines in the current image
    lane_lines = [] 
    for frame in buffer:
        inferred_lines = get_lanelines(frame,shape)
        lane_lines.append(inferred_lines)
        
    
    lane_lines = smoothen_the_lines(lane_lines)
    
    line_image  = np.zeros_like(buffer[0])
    
    for lane in lane_lines:
        lane.draw(line_image)
        
    masked_image = region_of_interest(line_image)
        
    combo_image = cv2.addWeighted(buffer[-1],0.8,masked_image,1,0)
    
    return combo_image


capture = cv2.VideoCapture("test2.mp4")
output_filename = "output.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
success,frame = capture.read()
shape = frame.shape
output_video = cv2.VideoWriter(output_filename,fourcc,20.0,(shape[1],shape[0]))

resize_h,resize_w = 540, 960



frame_buffer = deque(maxlen=5)

while capture.isOpened():
    success,frame = capture.read()
    if not success:
        break
    frame = cv2.resize(frame,(resize_w,resize_h))
    shape = frame.shape
    frame_buffer.append(frame)
    new_frame = process_image(frame_buffer,shape)
    output_video.write(new_frame)
    cv2.imshow("testvideo",new_frame)
    key = cv2.waitKey(1)
    if key == ord("s"):
        break
    
capture.release()
output_video.release()
cv2.destroyAllWindows()






QObject::moveToThread: Current thread (0x5608f6ec9660) is not the object's thread (0x5608f69bc850).
Cannot move to target thread (0x5608f6ec9660)

QObject::moveToThread: Current thread (0x5608f6ec9660) is not the object's thread (0x5608f69bc850).
Cannot move to target thread (0x5608f6ec9660)

QObject::moveToThread: Current thread (0x5608f6ec9660) is not the object's thread (0x5608f69bc850).
Cannot move to target thread (0x5608f6ec9660)

QObject::moveToThread: Current thread (0x5608f6ec9660) is not the object's thread (0x5608f69bc850).
Cannot move to target thread (0x5608f6ec9660)

QObject::moveToThread: Current thread (0x5608f6ec9660) is not the object's thread (0x5608f69bc850).
Cannot move to target thread (0x5608f6ec9660)

QObject::moveToThread: Current thread (0x5608f6ec9660) is not the object's thread (0x5608f69bc850).
Cannot move to target thread (0x5608f6ec9660)

QObject::moveToThread: Current thread (0x5608f6ec9660) is not the object's thread (0x5608f69bc850).
Cannot move to tar

In [ ]:

# Canny Edge Detection : Convert to Gradient Image 
def canny(image):
    # Convert to Gray Image 
    gray_image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    # Blur or Smoothen the Image to reduce noise 
    kernel_size = 5
    blur_image = cv2.GaussianBlur(gray_image,(kernel_size,kernel_size),0)
    # Apply Canny Edge Detection Algorithm :
    # Reject values below low_threshold and accept above the high_threshold 
    # value in between low and high are only accepted if the there neighbouring 
    # pixel is above high_threshold
    low_threshold = 50
    high_threshold = 150
    canny_image = cv2.Canny(blur_image,low_threshold,high_threshold)
    
    return canny_image

# Region Selection : Select the region of interest 
def region_selection(image):
    # Create a Black Image with same shape 
    mask = np.zeros_like(image)
    # Color of the region to be selected 
    shape_color = 255
    # Create a Shape around the lane lines 
    vertices = np.array([[(160,720),(680,380),(1200,720)]])
    # Draw shape on the mask image 
    cv2.fillPoly(mask,vertices,shape_color)
    # Crop the image to only the region of interest 
    masked_image = cv2.bitwise_and(image,mask)
    return masked_image


# Hough Transformation : Detect Edges or Lines  
def hough_transformation(image):
    # Define Parameter 
    rho = 2 
    theta = np.pi / 180
    threshold = 100
    min_line_len = 40
    max_line_gap = 5
    # Return list of lines detected 
    return cv2.HoughLinesP(image,rho,theta,threshold,np.array([]),min_line_len,max_line_gap)

def draw_lane_lines(image,lines):
    # Create a black image with same shape 
    lines_image = np.zeros_like(image)
    # Draw lines on the black image 
    if lines is not None :
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
#             print("Draw",x1,y1,x2,y2)
            cv2.line(lines_image,(x1,y1),(x2,int(y2)),(0,0,255),15)
    return lines_image

# Make the coordinates for slope and intercept 
def get_coordinates(image,line_parameters):
    print("Here",line_parameters)
    if line_parameters is not np.nan :
        slope , intercept = line_parameters
        # Get the shape of the image 
        shape = image.shape
        # Set the y coord 
        # Set the start point to the bottom of the image 
        # the end point to the 3/5 th of the image from the bottom 
        y1 = shape[0]
        y2 = int(y1 * (2/5))
        # y = mx + c => x = (y-c)/m 
        # Calculate the x coordinates 
        x1 = int((y1-intercept)/slope)
        x2 = int((y2-intercept)/slope)
        return np.array([x1,y1,x2,y2])
    else:
        return np.array([0,0,0,0])
    

def average_slopes(image,lines):
    left_fit = []
    right_fit = [] 
    # If some lines found
    if lines is  not None :
        for line in lines :
            x1,y1,x2,y2 = line.reshape(4)
#             print("Slope",x1,y1,x2,y2)
            # Calculate the slope and intercept of the coord 
            parameters = np.polyfit((x1,x2),(y1,y2),1)
            if parameters is not np.nan :
                slope, intercept = np.polyfit((x1,x2),(y1,y2),1)
                # if negative slope the line is on left lane 
                if slope < 0:
                    left_fit.append((slope,intercept))
                # otherise line is on right lane
                else:
                    right_fit.append((slope,intercept))
    # Calculate the average values for left and right lines
   
    left_fit_average = np.average(left_fit,axis=0)
    left_line = get_coordinates(image,left_fit_average)
    
    right_fit_average = np.average(right_fit,axis=0)
    right_line = get_coordinates(image,right_fit_average)
    
    return [left_line,right_line]

def get_candidate_lines(lines):
    candidate_lines = []
    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2),(y1,y2),1)
        if parameters is not np.nan :
            print(parameters)
            slope, intercept = np.polyfit((x1,x2),(y1,y2),1)
            if 0.5 <= np.abs(slope) <= 2:
                candidate_lines.append(line)
        
    return candidate_lines
    
# Algorithm for Lane Detection in an image 
def process_image(image):
    laneline_image = np.copy(image)
    canny_image = canny(laneline_image)
    cropped_image = region_selection(canny_image)
    lines = hough_transformation(cropped_image)
    candidate_lines = get_candidate_lines(lines)
    average_lines = average_slopes(image,candidate_lines)
    lines_image = draw_lane_lines(laneline_image,average_lines)
    combo_image = cv2.addWeighted(laneline_image,0.8,lines_image,1,0)
    return combo_image


capture = cv2.VideoCapture("test2.mp4")
output_filename = "output.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
success,frame = capture.read()
shape = frame.shape
output_video = cv2.VideoWriter(output_filename,fourcc,20.0,(shape[1],shape[0]))

# success,frame = capture.read()
# frame = process_image(frame)
# plt.imshow(frame)
# plt.show()


while capture.isOpened():
    success,frame = capture.read()
    if not success:
        break
    frame = process_image(frame)
    output_video.write(frame)
#     cv2.imshow("testvideo",frame)
#     key = cv2.waitKey(1)
#     if key == ord("s"):
#         break
    
capture.release()
output_video.release()
cv2.destroyAllWindows()
